In [ ]:
# imports
from openai import OpenAI
import json

In [ ]:
#API key
with open('key') as k:
    OPENAI_API_KEY=k.read().strip()

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
SYSTEM_PROMPT="""
You are an assistant who can help the user to find out what are the best parameters like number of MPI processes, number of OMP threads, and GPU use for different
subtasks of molecular dynamics simulation using Gromacs on a particular protein which structure can be downloaded from the PDB database.
Assume the user quite expert in computational biochemistry but he/she does not understand the details of computational setup, and uses this as a blackbox,
just copy and pasting Gromacs command line parameters.
"""

In [ ]:
def gmxtune(pdbid):
    """
    Download initial PDB structure, set up Gromacs simulation and, by talking to the tuner service, find out what are the most promising
    combinations of parameters like number of MPI processes, number of OMP threads, and GPU use for different
    subtasks of molecular dynamics.

    Args:
        pdb (str): The identifier of the input structure to be found in PDB database
    
    Returns:
        str: List of the most promising gromacs commands to execute, including the estimated performance (the more, the better)
    """
    
    return f"""gmx mdrun -ntomp 2 -ntmpi 4 -nb gpu -pme gpu -deffnm {pdbid} # 222 ns/day
gmx mdrun -ntomp 4 -ntmpi 4 -pme gpu -deffnm {pdbid} 188 ns/day
"""

In [ ]:
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "What is the best way to run Gromacs molecular dynamics simulation on PDB id 1FME" }
]

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

In [ ]:
print(response.choices[0].message.content)

In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": gmxtune.__name__,
        "description": gmxtune.__doc__,
        "parameters": {
            "type": "object",
            "properties": {
                "pdbid": {"type": "string"},
            },
            "required": ["pdbid"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

In [ ]:
response

In [ ]:
messages.append(response.choices[0].message)

In [ ]:
pdbid = eval(response.choices[0].message.tool_calls[0].function.arguments)['pdbid']
pdbid

In [ ]:
messages.append(
    {
    "role": "tool",
    "tool_call_id": response.choices[0].message.tool_calls[0].id,
    "content": gmxtune(pdbid)
    })
messages

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

In [ ]:
response

In [ ]:
print(response.choices[0].message.content)